 Remove group dictionary and single letter word

# Text Clean Up

### **Changelog**:

**November 18, 2022**
> Changed the structure of the cleanup pipeline

> Under extra_cleans()
* Changed '\b' to '\s' in `d **bold text**elete_single_character_except_ai` at the start to prevent deleting contractions e.g.(don't, isn't)
* Changed `/` within words with `or` e.g.(groups/teams --> groups or teams) 
* Removed remaining `/` e.g.(The technical growth/. --> The technical growth)

> Under rem_expressions()
* replaced {2,} with {3,} to prevent deleting words such as heir in "their" or
any words that starts with h and followed by 2 vowels e.g.(hair, hierarchical)

> Combined lang_detect() & translate() to translate_lang() function
* Changed the default threshold to 99

> Made mark_none() function to tag those **na**, **none**, and **nope** feedbacks then remove them

> Removed 1 letter word that is either `Yes` or `No`

**December 09, 2022**
> Included Timestamp with `YYYY-MM` format



## Package Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%capture
!pip install fasttext googletrans==3.1.0a0 spacy-transformers emoji
!python -m spacy download en_core_web_trf

In [ ]:
import spacy_transformers
import emoji
from collections import Counter
from wordcloud import WordCloud
import pandas as pd
import numpy as np 
# For language confidence detection
import fasttext #pip install fasttext
from googletrans import Translator # translate non english words 
import spacy
import spacy_transformers
import re
import seaborn as sns 
import matplotlib.pyplot as plt
import nltk
nltk.download('words')
from nltk.corpus import words
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

wb = gc.open_by_key('1bD-IYVdxwPdNhNc9SQIZAsv36l7tqkaZ9aHrE3Mgm7E')

ws = wb.worksheet('Exit Interview Responses')

# get_all_values gives a list of rows.
rows = ws.get_all_values()
# print(rows)

## Load DataFrame from Google Sheets

In [ ]:
pd.set_option('display.max_columns', None)

# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows[2:], columns = rows[1])
df.Timestamp = pd.to_datetime(df.Timestamp)
print('Size of DataFrame:', df.shape)
print('DataFrame Columns:', df.columns)
df.head()

Size of DataFrame: (587, 74)
DataFrame Columns: Index(['DUMMY ID NO', 'Timestamp', 'Name of your Group',
       'What prompted you to leave the Company? ',
       'Before making a decision to leave, did you investigate other options that would enable you to stay? If your answer is YES, please describe using the next space. ',
       '',
       'Were your objectives for joining this Company met? Please elaborate on how they were met or how they may have changed.',
       'Type of work', 'Fairness of workload', 'Salary', 'Benefits',
       'Working Conditions', 'Tools/Equipment', 'Trainings', 'Co-workers',
       'Supervision', 'Department cooperation', 'Inter-Department cooperation',
       'Communications - Company', 'Communications - Manager',
       'Morale of Department', 'Job Satisfaction', 'Growth Potential',
       'Was your workload usually: ',
       'What did you like MOST about your job and/or this Company?',
       'What did you like LEAST about your job and/or this Company?

,DUMMY ID NO,Timestamp,Name of your Group,What prompted you to leave the Company?,"Before making a decision to leave, did you investigate other options that would enable you to stay? If your answer is YES, please describe using the next space.",,Were your objectives for joining this Company met? Please elaborate on how they were met or how they may have changed.,Type of work,Fairness of workload,Salary,Benefits,Working Conditions,Tools/Equipment,Trainings,Co-workers,Supervision,Department cooperation,Inter-Department cooperation,Communications - Company,Communications - Manager,Morale of Department,Job Satisfaction,Growth Potential,Was your workload usually:,What did you like MOST about your job and/or this Company?,What did you like LEAST about your job and/or this Company?,How do you feel about your salary and employee benefits? [Base Pay],How do you feel about your salary and employee benefits? [Allowances],How do you feel about your salary and employee benefits? [Bonuses],How do you feel about your salary and employee benefits? [HMO/Medical/Dental Plan],How do you feel about your salary and employee benefits? [Group Term Life Insurance],How do you feel about your salary and employee benefits? [Vacation Leave],How do you feel about your salary and employee benefits? [Sick Leave],How do you feel about your salary and employee benefits? [Paid Time-Off],How do you feel about your salary and employee benefits? [Maternity Benefit],How do you feel about your salary and employee benefits? [Bereavement Leave],Are there any other benefits you feel should have been offered? Please provide details.,Please rate the following aspects of the ORGANIZATION. [Rewards & Recognition],Please rate the following aspects of the ORGANIZATION. [Performance Management],Please rate the following aspects of the ORGANIZATION. [Organizational Structure],Please rate the following aspects of the ORGANIZATION. [Policies and Work Processes],Please rate the following aspects of the ORGANIZATION. [Communication],Please rate the following aspects of the ORGANIZATION. [Leadership Practices],Please rate the following aspects of the ORGANIZATION. [Talent Acquisition],Please rate the following aspects of the ORGANIZATION. [Learning/Employee Growth],Please rate the following aspects of the ORGANIZATION. [Career Development],Please rate the following aspects of the ORGANIZATION. [Employee Programs],Please rate the following aspects of the ORGANIZATION. [Safety and Security],How frequently did you get performance feedback?,What were your feelings about the performance review process?,How frequently did you have discussions with your manager about your career goals?,Is there anything you would like to suggest to make the Company's system effective?,Please rate your Immediate Superior. [Hires and develops the people in his/her team],Please rate your Immediate Superior. [Provides appropriate and challenging assignments],Please rate your Immediate Superior. [Clearly communicates directions and expectations],Please rate your Immediate Superior. [Challenges the status quo and encourages creativity],"Please rate your Immediate Superior. [Gives regular, timely and actionable performance feedback]",Please rate your Immediate Superior. [Recognizes and rewards work done well],Please rate your Immediate Superior. [Genuinely cares for his/her team],Please rate your Immediate Superior. [Promotes and encourages work-life balance],Please rate your Immediate Superior. [Honors commitment he/she has made],Please rate your Immediate Superior. [Demonstrates fair and equal treatment],Please rate your Immediate Superior. [Develops cooperation and teamwork],Please rate your Immediate Superior. [Encourages/listens to suggestions],Please rate your Immediate Superior. [Resolves complaints and problems],Please rate your Immediate Superior. [Follows policies and practices],Do you think your Immediate Superior is a good leader? Why or Why not (use the space below)?,,What would you have differentl

### Select Free Text Columns (Questions) + Meta Information (Dummy ID, Timestamp, Group Name)

In [ ]:
def column_cleanup(col):
    # Strip text of whitespace from both ends
    col = col.strip()
    return col if len(col)!=0 else np.nan

question_cols = [
    'What prompted you to leave the Company?',
    'Before making a decision to leave, did you investigate other options that would enable you to stay? If your answer is YES, please describe using the next space.',
    'What did you like MOST about your job and/or this Company?',
    'What did you like LEAST about your job and/or this Company?',
    "Is there anything you would like to suggest to make the Company's system effective?",
    'What constructive comments would you give to Management to make this Company a better employer?',
    'Are there any other benefits you feel should have been offered? Please provide details.',
    'Do you think your Immediate Superior is a good leader? Why or Why not (use the space below)?',
    'What would you have differently if you were the manager?',
    'Were your objectives for joining this Company met? Please elaborate on how they were met or how they may have changed.',
    'What were your feelings about the performance review process?',
    'Would you work for this Company again if there is an opportunity? What circumstances/conditions (if there are any) will make you accept again a job offer with this Company?',
    'Please use this space for additional comments about your job or this Company.',
    'How frequently did you get performance feedback?',
    'How frequently did you have discussions with your manager about your career goals?',
    'Is there a point of uncertainty or disagreement that you have been unable to settle unsatisfactorily that you would like to discuss?',
]

cols = ['DUMMY ID NO',  'Timestamp', 'Name of your Group'] + question_cols
df.columns = df.columns.map(column_cleanup)

null_column_idx = np.where(df.columns.isna())[0] # find index of null columns

df.columns = df.columns.to_series().ffill()
df = (df.iloc[:, np.delete(np.arange(df.shape[1]), null_column_idx-1)]
        .loc[:, cols])
df

,DUMMY ID NO,Timestamp,Name of your Group,What prompted you to leave the Company?,"Before making a decision to leave, did you investigate other options that would enable you to stay? If your answer is YES, please describe using the next space.",What did you like MOST about your job and/or this Company?,What did you like LEAST about your job and/or this Company?,Is there anything you would like to suggest to make the Company's system effective?,What constructive comments would you give to Management to make this Company a better employer?,Are there any other benefits you feel should have been offered? Please provide details.,Do you think your Immediate Superior is a good leader? Why or Why not (use the space below)?,What would you have differently if you were the manager?,Were your objectives for joining this Company met? Please elaborate on how they were met or how they may have changed.,What were your feelings about the performance review process?,Would you work for this Company again if there is an opportunity? What circumstances/conditions (if there are any) will make you accept again a job offer with this Company?,Please use this space for additional comments about your job or this Company.,How frequently did you get performance feedback?,How frequently did you have discussions with your manager about your career goals?,Is there a point of uncertainty or disagreement that you have been unable to settle unsatisfactorily that you would like to discuss?
0,D100088,2019-01-09 15:25:29,Mynt Technology Corps,"Compensation, Quality of Supervision, Career A...",,Team and Product,Leadership,Review the business cycle and,They're already in the right direction,NA,Hero mentality syndrome,Proper work ethics and gives trust,Career advancement - knowledge wise,not aware,Once the structure is better,,Rare,once,No
1,D101431,2019-01-10 09:12:01,Business Operations and Customer Experience,"Lack of Recognition, Quality of Supervision",I tried to apply as a Fraud Analyst for Fraud ...,I accomplished so much more than I ever though...,Working / Supporting on weekends. The need to ...,I would suggest to look after each Departments...,•\tCommunicate and appreciate workers.\n•\tFoc...,None,Because her actions does not inspire others to...,I will not follow a dictatorial style of leade...,Yes. Because I learned something new and enhan...,"Honestly, I did not experienced a formal PPE o...",Definitely yes! at any circumstance or condition.,"I want to say thank you to YOU, the management...",Not often.,Not often.,"Yes, there is."
2,D100216,2019-01-11 14:50:57,Enterprise,quality of the product being offered which res...,"yes, given the organizational change I weighed...",My colleagues and the passion I see in my IS a...,The GCash p3 - the system,The processes outside each department and units.,"Simplify the process, Improve the products (te...",,She is a good leader who's leadership style is...,Be more strict on some members of the team who...,Yes as I have learned new technical stuffs but...,I feel good since I know where I stand. I know...,definitely,,regularly,Timely enough,no
3,D100311,2019-01-11 16:29:58,"Information Security, Privacy, and Continuity",Location,,You get to work with young leaders that would ...,"Office Politics, corporate policy are not clea...","Invest on automation (i.e. HR Tool, Ticketing ...",Support each others projects or program. Live ...,"14th month, Dental reimbursement on top of the...",She can be a good leader should she have more ...,I would start hiring more resources to ensure ...,"In terms of career development, my objectives ...",No setting of objectives that happened and no ...,YES,,Very rare. Unless there's a PPE or and issue w...,We seldom have discussion regarding my career ...,None
4,D100587,2019-01-17 13:07:40,"Risk, Data, and Intelligence",Family Circumstances,If I can continue working in the Philippines (...,I like how my job/role allowed me to be more c...,I find the company's lack of structure and pro...,Th

### Defining  Constants and Dictionaries

#### Question Tagging Dictionary

In [ ]:
QUESTION_TAGGING = {q: f'Q{i+1}' for i, q in enumerate(question_cols)}

df = df.rename(columns=QUESTION_TAGGING)
df['year'] = df.Timestamp.dt.year
df['month'] = df.Timestamp.dt.month
df

,DUMMY ID NO,Timestamp,Name of your Group,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,year,month
0,D100088,2019-01-09 15:25:29,Mynt Technology Corps,"Compensation, Quality of Supervision, Career A...",,Team and Product,Leadership,Review the business cycle and,They're already in the right direction,NA,Hero mentality syndrome,Proper work ethics and gives trust,Career advancement - knowledge wise,not aware,Once the structure is better,,Rare,once,No,2019,1
1,D101431,2019-01-10 09:12:01,Business Operations and Customer Experience,"Lack of Recognition, Quality of Supervision",I tried to apply as a Fraud Analyst for Fraud ...,I accomplished so much more than I ever though...,Working / Supporting on weekends. The need to ...,I would suggest to look after each Departments...,•\tCommunicate and appreciate workers.\n•\tFoc...,None,Because her actions does not inspire others to...,I will not follow a dictatorial style of leade...,Yes. Because I learned something new and enhan...,"Honestly, I did not experienced a formal PPE o...",Definitely yes! at any circumstance or condition.,"I want to say thank you to YOU, the management...",Not often.,Not often.,"Yes, there is.",2019,1
2,D100216,2019-01-11 14:50:57,Enterprise,quality of the product being offered which res...,"yes, given the organizational change I weighed...",My colleagues and the passion I see in my IS a...,The GCash p3 - the system,The processes outside each department and units.,"Simplify the process, Improve the products (te...",,She is a good leader who's leadership style is...,Be more strict on some members of the team who...,Yes as I have learned new technical stuffs but...,I feel good since I know where I stand. I know...,definitely,,regularly,Timely enough,no,2019,1
3,D100311,2019-01-11 16:29:58,"Information Security, Privacy, and Continuity",Location,,You get to work with young leaders that would ...,"Office Politics, corporate policy are not clea...","Invest on automation (i.e. HR Tool, Ticketing ...",Support each others projects or program. Live ...,"14th month, Dental reimbursement on top of the...",She can be a good leader should she have more ...,I would start hiring more resources to ensure ...,"In terms of career development, my objectives ...",No setting of objectives that happened and no ...,YES,,Very rare. Unless there's a PPE or and issue w...,We seldom have discussion regarding my career ...,None,2019,1
4,D100587,2019-01-17 13:07:40,"Risk, Data, and Intelligence",Family Circumstances,If I can continue working in the Philippines (...,I like how my job/role allowed me to be more c...,I find the company's lack of structure and pro...,The leadership structure may be improved by al...,If management can get a better feel of the nee...,It would be nice to have convertible leaves. I...,I know that he has the potential to be a great...,I'd like to be more aware of the development o...,Somehow. One of the objectives I had upon join...,"I think it's okay. For improvement, it would b...",Yes. Given the right role and a clearer vision...,,i get feedback on my performance every time we...,We've had a discussion on career goals wheneve...,None,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,D101223,2022-11-07 09:43:35,TECHNOLOGY AND OPERATIONS,"Work Conditions, Family Circumstances, Career ...",My Supervisor give me some option but the my f...,Events,Workload,Proper implementation of agile if that is what...,Don't overload/overwork your employee,NA,he demonstrate how to be a good leader,"None, I think he is the best","yes, but I need to get a lot of compensation s...",NA,"Yes, better workload","for now, not so, because I don't want them to ...",NA,NA,NA,2022,11
583,D101159,2022-11-07 12:59:50,TECHNOLOGY AND OPERATIONS,"Family Circumstances, Career Advancement Oppor...",My immediate supervisor did his best to ask to...,The work collaboration I have with my Immediat...,If there is something I like least about what ..

In [ ]:
Q_DICT = {v:k for k, v in QUESTION_TAGGING.items()}
Q_DICT

{'Q1': 'What prompted you to leave the Company?',
 'Q2': 'Before making a decision to leave, did you investigate other options that would enable you to stay? If your answer is YES, please describe using the next space.',
 'Q3': 'What did you like MOST about your job and/or this Company?',
 'Q4': 'What did you like LEAST about your job and/or this Company?',
 'Q5': "Is there anything you would like to suggest to make the Company's system effective?",
 'Q6': 'What constructive comments would you give to Management to make this Company a better employer?',
 'Q7': 'Are there any other benefits you feel should have been offered? Please provide details.',
 'Q8': 'Do you think your Immediate Superior is a good leader? Why or Why not (use the space below)?',
 'Q9': 'What would you have differently if you were the manager?',
 'Q10': 'Were your objectives for joining this Company met? Please elaborate on how they were met or how they may have changed.',
 'Q11': 'What were your feelings about the

In [ ]:
lens = [len(i.split()) for ii in df.columns[3:19] for i in df[ii].to_list()]

In [ ]:
sum(lens)

144219

In [ ]:
lens

[7,
 6,
 14,
 1,
 2,
 6,
 3,
 3,
 14,
 15,
 3,
 8,
 6,
 2,
 2,
 3,
 3,
 11,
 2,
 8,
 2,
 6,
 6,
 7,
 3,
 3,
 3,
 6,
 13,
 2,
 3,
 3,
 9,
 3,
 10,
 1,
 5,
 9,
 3,
 9,
 5,
 7,
 6,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 3,
 5,
 17,
 9,
 2,
 5,
 1,
 3,
 9,
 6,
 7,
 3,
 8,
 6,
 5,
 9,
 1,
 14,
 3,
 453,
 2,
 4,
 6,
 2,
 19,
 8,
 2,
 4,
 5,
 5,
 6,
 5,
 3,
 7,
 115,
 7,
 2,
 3,
 7,
 3,
 8,
 2,
 5,
 6,
 10,
 9,
 5,
 14,
 2,
 3,
 3,
 5,
 14,
 1,
 2,
 14,
 6,
 8,
 6,
 15,
 5,
 10,
 9,
 3,
 9,
 2,
 6,
 4,
 2,
 174,
 7,
 9,
 4,
 2,
 6,
 15,
 15,
 6,
 9,
 6,
 2,
 3,
 8,
 3,
 11,
 22,
 9,
 10,
 5,
 7,
 11,
 6,
 5,
 4,
 9,
 2,
 3,
 10,
 5,
 3,
 8,
 30,
 3,
 4,
 5,
 6,
 3,
 9,
 16,
 13,
 2,
 9,
 3,
 5,
 138,
 4,
 9,
 4,
 3,
 2,
 9,
 7,
 3,
 4,
 3,
 6,
 15,
 9,
 3,
 17,
 7,
 5,
 11,
 1,
 6,
 9,
 31,
 8,
 1,
 3,
 9,
 6,
 4,
 3,
 6,
 3,
 7,
 2,
 14,
 2,
 33,
 13,
 5,
 9,
 6,
 13,
 5,
 5,
 4,
 7,
 2,
 9,
 7,
 7,
 3,
 6,
 4,
 7,
 11,
 7,
 13,
 17,
 3,
 7,
 16,
 4,
 11,
 5,
 3,
 3,
 3,
 9,
 6,
 6,
 4,
 2,
 9

In [ ]:
qwe = []

for ii in df['Q6']:
    qwe.append(len(ii.split()))

In [ ]:
sum(qwe)

14521

In [ ]:
df.columns[3:19][0]

'Q1'

In [ ]:
df = df.loc[548:]
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,DUMMY ID NO,Timestamp,Name of your Group,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,year,month
0,D101149,2022-08-23 12:01:42,PEOPLE & PURPOSE,"Family Circumstances, WORK SET UP & FLEXIBILITY",I had several conversations/discussions with G...,Very dynamic and fast paced - I was given the ...,Constant change in direction for certain proje...,TM should communicate the deadlines ahead of t...,Flexibility for remote/WFH work for working pa...,Option to add senior dependents through salary...,Gami is one of those leaders who has empathy &...,Nothing. I'm very happy with Gami as my IS :),In the beginning my role was confusing because...,Generally I feel like it lacks in transparency...,"Definitely. I was/am happy with my role, my re...",I really enjoy doing Talent Marketing for GCas...,I received regular feedback from Gami & Robert...,Regularly / frequently. Gami and I would sched...,None at the moment,2022,8
1,D100418,2022-08-24 09:35:21,COMMERCIAL,Family Circumstances,- Tried to manage situation at home as so that...,Culture - my ideas are valued and heard even b...,Lack of control on items that would impact my ...,None,Need to fill headcount so that people are not ...,,Ferdie is a very good and smart leader. I thin...,I would check up more on the team and boost mo...,"Yes, had a ton of learning.",Average,Yes. Given that my issues at home are resolved...,,"Previously with my original IS, very frequentl...",Very few instances with new IS.,None,2022,8
2,D101602,2022-08-26 21:20:32,FINANCE,"Compensation, Career Advancement Opportunity",Higher basic salary,"I am given the chance to give feedback, sugges...",Laptop. It takes a while before you get a lapt...,More company and team activities like outings ...,"I think, make sure that people have enough res...",retirement funds and carplan,My supervisor always checks up on us if we are...,None,Yes. I was given a permanent position and got ...,my supervisors are honest and treated me kindl...,Absolutely. I love working in Gcash.,It has been a blissful 5 years Gcash. I am and...,twice a year,Twice a year,None,2022,8
3,D100098,2022-08-30 17:53:41,COMMERCIAL,"Compensation, Company Culture, Career Advancem...",,"I really like the purpose of GCash ""Finance fo...",No equal opportunity for the tenure employee t...,There should be an internal hiring for some po...,Give value to your tenured employee who works ...,"Salary Increase, Loyalty Program, Retirement B...",Rating my IS with 4 because he almost always d...,"Equal treatment, sympathy and empathy",Yes. As a pioneer employee of GCash in Mindana...,Not materializing. Been doing my IDP for 6year...,Maybe.,,Twice a year - H1 and H2,Twice a year - H1 and H2,NONE,2022,8
4,D100783,2022-08-31 14:16:19,RISK,Family Circumstances,I enjoy what I do. I like my manager's leaders...,My manager and her leadership.,Slow recruitment process which results to peop...,Invest in HR (recruitment) and make company's ...,I'm grateful for the leadership experience ent...,,"She knows how to listen, how to teach people, ...",My manager is almost perfect but she has nothi...,"Yes, I learned a lot. My manager helps me to b...",fair,"Yes, if there's a right manpower.",GCash is a great company and has amazing benef...,monthly,almost every month,N/A,2022,8
5,D101477,2022-09-05 17:40:48,COMMERCIAL,"Business/Product Direction, Focus on business",,What I like about my job is really the exposur...,"None really, but I think there can be an impro...",A more structured and timed training module f...,Have a better training even for experienced hi...,Car Plan.,"Yes, our IS is a good leader, he really pushes...",I will have to be more empathizing and sympath...,"Yes, my objective of learning about distributi...",The performance review are objective.,"Yes, definitely! different assignment more ali...",,Monthly,Monthly,None so far,2022,9
6,D101398,2022-09-12 13:34:24,CUSTOMER,Type of Work,Movement of different team --> mkt to HR,People,The execution role,Lessen manual work,R

# Building Text Clean Up Pipeline


In [ ]:
# def get_paragraphs(text): 
#     paras = re.findall(r'\w+.*(?=\n?)', text)

In [ ]:
# find_sentences = r'[0-9]{2,}[a-zA-Z].*|[a-zA-Z].*' # You cannot remove the numbers in the beginning because m

# text = """
# 1) sadilgnesiangsepingipean. asibnvaipnvgpoan, 2) daignaepigna

# a) dsgskngskrngsr

# - af;an;ang;

# - af;an;ang;aeg, 

# •\t sknkngk

# 2) saipgnaeipngaesping.

# 5 times total, I think?

# 13-15 month pay!
# """
# re.findall(find_sentences, text), re.findall(r'\w+.*(?=\n?)', text)

In [ ]:
# x = '5 times total, I think! ; [hehe]!]\n'

# x.strip().endswith(tuple(',;.?!\'"}'))

# # x.strip().endswith(tuple([',', ';', '.', '!', '?']))

# # x.strip().endsw

In [ ]:
# pars = re.findall(r'\w+.*(?=\n?)', text)




# [par.strip()+'.' if not par.strip().endswith(tuple('.,;?!')) else par.strip() for par in pars]

In [ ]:
test = '''I would like to thank our Mynt Barkada for all the opportunities that was given to me learn more about:
the industry, the business, the technology and the chance to help develop a cashless nation that will lead to finance; 
for all.  I will forever be proud that even for a short time I was part of a dynamic company who has a very noble purpose.  Please never forget this purpose, as it will ensure the company's success.  I will always believe that there is no such thing as good bye... only see you later... See you around Mynt Barkada! :-)
'''

In [ ]:
paragraphs = get_paragraphs(test)
paragraphs

NameError: ignored

In [ ]:
define_sentences(paragraphs)

NameError: ignored

In [ ]:
## Delete emoji / expressions first [cause we have sentences ending with :), :)), :-), :(]
## then apply get_paragraphs() and define_sentences

In [ ]:
type(Q_DICT.keys())

dict_keys

## Load pre-trained models

Load `fasttext` language identification model:

Supports 175 languages including:
* **tl** - Tagalog 
* **en** - English

Reference:
https://fasttext.cc/docs/en/language-identification.html

In [ ]:
# for language detection
lang_model = fasttext.load_model('/content/drive/Shareddrives/R&D Innovations/HR Analytics/Topic Modeling/lid.176.bin')

Load Language Processing Pipeline in `Spacy`

Reference: https://spacy.io/models/en

Load Custom Model

In [ ]:
nlp = spacy.load('/content/drive/Shareddrives/R&D Innovations/HR Analytics/Topic Modeling/ner_model')

This function will will use `fasttext` to detect the english words distribution in a document. It either retains a text depending on the threshold or a text already translated when the fasttext score is less than the threshold.
 * threshold is 99% by default
 * if fastest score is >= the threshold, then retain the doc otherwise it will be subject for translation

> Only **caveat** here is short text sentences e.g. ('Family Circumstances', 'Lack of Recognition') gives a low fasttext english score. But rest assured this words will not change during translation since it's already english.

This function will translate the list from `translate_lang` (the function above) to english words. And return a list of translated docs via `doc_translated` which will be appended to the `retain` (the function above) later on.


## rem_expressions()

`clean_expressions` - Delete or remove words e.g. (***haha, hehe, hihi, hoho, huhu***) even (***hhhahahhah, hehehhehheh***), also (***lol***)



In [ ]:
def rem_expressions(texts):
    emoj_notations = [':)', ':))', ':(', ':-)', '<3']

    laugh_express = [
                    r'\b[h|a|e|i|o|u|H|A|E|I|O|U][h|a|e|i|o|u|H|A|E|I|O|U][h|a|e|i|o|u|H|A|E|I|O|U]{2,}\b',
                    r'\b[l|L][o|O][l|L]{1,}\b'
                    ]

    emoji_notation = []

    emojis = emoji.distinct_emoji_list(texts)

    doc = nlp(texts)
    emoj_notation_toc = [token.text for token in doc]
    
    emoj_notation = [emoj for emoj in emoj_notation_toc if emoj in emoj_notations]

    for i in emoj_notation:
        if len(i) == 2:
            emoji_notation.append("\\"+ i[len(i)-2] + "\\" + i[len(i)-1])
        elif len(i) == 3:
            emoji_notation.append("\\"+ i[len(i)-3] + "\\" + i[len(i)-2] + "\\" + i[len(i)-1])

    laugh_expressions = re.findall('|'.join(laugh_express), texts)

    del_emojis = emojis + emoji_notation + laugh_expressions

    sentence = re.sub('|'.join(del_emojis), '', texts)

    return sentence

## Extra Cleans

In [ ]:
# add . if % ) ] ' ( "

# change to . if ;

# leave alone if : ,  & ! ?

In [ ]:
def get_paragraphs(text):
    return re.findall(r'\w+.*(?=\n?)', text)

def define_sentences(text_list):
    sentences = []
    for par in text_list:
        x = par.strip()
        if not x.endswith(tuple('.,:;&!?')):
            sentences.append(x+'.')
        elif x.endswith(';'):
            sentences.append(re.sub(r';$', '.', x))
        else:
            sentences.append(x)
    return ' '.join(sentences)

def clean_structure(text):
    clean_double_spaces = re.sub(r'\s{2,}', ' ', text)

    return clean_double_spaces

## yn_variants()

In [ ]:
def yn_variants(texts):

    yn_variants = [
                r'^[yY][e|E][s|S]$', # yes
                r'^[yY][e|E][s|S]\W$', # yes,
                r'^[yY][e|E][s|S]\W\W$', # yes,,
                r'^[n|N][o|O]$', # no
                r'^[n|N][o|O]\W$', # no,
                r'^[n|N][o|O]\W\W$'
                ]

    find = re.findall('|'.join(yn_variants), texts)
    if len(find) != 0:
        return 'has yn'
    else:
        return 'valid'

## null_variants()

> Apparently some feedbacks contains **None**, **Nope**, **Na**, **N/a**, and **N\a**. So deleting them already would kinda help for topic modeling using regex.
* re.findall() was used and if the `find_none` has a length greater than 0, then it means it contains the words listed above, otherwise we mark it as valid

In [ ]:
def null_variants(texts):

    null_variants = [
                    r'^[n|N]$' # n
                    r'^[n|N]\/[a|A]$', # n\a
                    r'^[n|N]\/[a|A]\.$', # n\a.
                    r'^[n|N]\\[a|A]$', # n\a
                    r'^[n|N]\\[a|A]\.$', # n/a.
                    r'^[n|N][a|A]$', # na
                    r'^[n|N][a|A]\.$', # na.
                    r'^[n|N].[a|A]\.$', # n.a.
                    r'^[n|N]ope$', # nope
                    r'^[n|N]ope\.$', # nope.
                    r'^[n|N]one', # starts with none
                    r'^[n|N]othing', # starts with nothing

                    r'^\w$', # match single word char
                    r'^\W$', # match singe non-word char

                    r'[n|N]one\sthat', # contains none that
                    r'[c|C]an\'t\sthink', # contains can't think
                    r'[c|C]ant\sthink', # contains cant think
                    r'[c|C]annot\sthink', # contains cannot think
                    r'^[i|I]\scan\'t\sreally', # starts with i can't really
                    r'^[i|I]\scant\sreally', # starts with i cant really 
                    r'[n|N]o\scomment', # contains no comment
                    r'^[i|I]\sdon\'t\sknow', # starts with i don't know
                    r'^[i|I]\sdont\sknow', # starts with i dont know
                    r'^[n|N]ot\sapplicable', # starts with not applicable
                    r'^[s|S]o\sfar\W\snone$', # so far, none
                    r'^[s|S]o\sfar\snone$' # so far none
                    ]

    find = re.findall('|'.join(null_variants), texts)
    if len(find) != 0:
        return 'has null'
    else:
        return 'valid'

## tl_spelling()

In [ ]:
def tl_spelling(text):
    
    lang = re.sub(r'\b[a|A][l|L][n|N][g|G]\b|\b[l|L][n|N][g|G]\b', 'lang', text)
    niya = re.sub(r'\b[n|N][i|I][a|A]\b|\b[n|N][y|Y][a|A]\b', 'niya', lang)
    iyong = re.sub(r'\b[u|U][n|N][g|G]\b|\b[y|Y][u|U][n|N][g|G]\b', 'iyong', niya)
    siya = re.sub(r'\b[c|C][y|Y][a|A]\b|\b[s|S][y|Y][a|A]\b|\b[s|S][i|I][a|A]\b', 'siya', iyong)
    ninyo = re.sub(r'\b[n|N][i|I][o|O]\b|\b[n|N][y|Y][o|O]\b|\b[n|N][i|I][y|Y][o|O]\b|\b[n|N][i|I][u|U]\b', 'ninyo', siya)
    noong = re.sub(r'\b[n|N][u|U][n|N][g|G]\b', 'noong', ninyo)
    iyon = re.sub(r'\b[y|Y][u|U][n|N]\b|\b[y|Y][o|O][n|N]\b', 'iyon', noong)
    kapag = re.sub(r'\b[k|K][p|P][a|A][a|A][g|G]\b|\b[k|K][p|P][a|A][g|G]\b', 'kapag', iyon)
    because = re.sub(r'\b[c|C][o|O][z|Z]\b|\b[c|C][o|O][s|S]\b', 'because', kapag)

    return because

In [ ]:
# tl_spelling('lng alng nya nia ung yung cya sya sia nio nyo niyo niu nung yun yon kpaag kpag coz cos')

## translate_lang()

In [ ]:
# Function for fasttext language detection

def translate_lang(texts, threshold=99):
    sent_translated = []
    translator = Translator()

    doc = nlp(texts)

    fasttext_score = lang_model.predict(texts, k=1)[1][0]*100
    if fasttext_score >= threshold:
        return texts
    else:
        for sents in doc.sents:
            out = translator.translate(sents.text, dest='en', src='tl')
            sent_translated.append(out.text)
            translated = ' '.join(sent_translated)
        return translated

## detect_person()

In [ ]:
exclude_detect = ['loa', 'pace', 'mancom', 'job', 'reorg', 'silos', 'agile without backbone', 'jd', 'jira', 'prio',
                  'si great', 'open-minded', 'izfine', 'ppe', 'jobs', 'job\'s', 'christ'
                 ]

In [ ]:
det_low = [ex for ex in set(exclude_detect)]
det_cap = [ex.capitalize() for ex in set(exclude_detect)]
det_title = [ex.title() for ex in set(exclude_detect)]
det_upper = [ex.upper() for ex in set(exclude_detect)]

In [ ]:
exclude_detect = det_low + det_cap + det_title + det_upper
len(exclude_detect)

68

In [ ]:
def detect_person(text, exclude):

    person_count = []

    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            person_count.append(ent.text)

    for per in person_count:
        if per in exclude:
            person_count.remove(per)
    
    if len(person_count) == 0:
        return text.lower()
    else:
        sentence = re.sub('|'.join(person_count), 'name_tag', text)
        return sentence.lower()

In [ ]:
test = '''Micro manage in the form of Jira	 
'''

In [ ]:
detect_person(test, exclude_detect)

'micro manage in the form of jira\t \n'

## word_uniformity()

This function deals with replacing some values for uniformity e.g. (groups, immediate supervisors, managers)
* `new_group_value` - Using the **replace_all** function above to replace group names with respective values.
* 'new_is_value` - Deals with lowercase/uppercase e.g.(**IS, immediate supervisor[s], immediate_superior[s], supervisor[s], superior[s]**) and replace it with ***immediate_supervisor***.
* 'new_manager_value` - Deals with lowercase/uppercase e.g.(**manager[s], manager/[s], manager'[s]**) and replace it with ***manager***.

In [ ]:
# Function for changing IS, Manager

def word_uniformity(text):

    new_is = [
            r'\b[I|i][M|m]{2}[e|E][d|D][i|I][a|A][t|T][e|E]\s[S|s][u|U][p|P][e|E][r|R][v|V][i|I][s|S][o|O][r|R][s|S]\b',
            r'\b[I|i][M|m]{2}[e|E][d|D][i|I][a|A][t|T][e|E]\s[S|s][u|U][p|P][e|E][r|R][v|V][i|I][s|S][o|O][r|R]\b',
            r'\b[I][S]\'[s|S]\b',
            r'\b[I][S]\b',
            r'\b[I][S][s|S]\b',
            r'\b[s|S][u|U][p|P][e|E][r|R][i|I][o|O][r|R]\'[s|S]\b',
            r'\b[s|S][u|U][p|P][e|E][r|R][i|I][o|O][r|R][s|S]\b',
            r'\b[s|S][u|U][p|P][e|E][r|R][i|I][o|O][r|R]\b',
            r'\b[s|S][u|U][p|P][e|E][r|R][v|V][i|I][s|S][o|O][r|R]\'[s|S]\b',
            r'\b[s|S][u|U][p|P][e|E][r|R][v|V][i|I][s|S][o|O][r|R][s|S]\b',
            r'\b[s|S][u|U][p|P][e|E][r|R][v|V][i|I][s|S][o|O][r|R]\b',
            ]

    new_manager = [
                r'[m|M][a|A][n|N][a|A][g|G][e|E][r|R]\'[s|S]',
                r'[m|M][a|A][n|N][a|A][g|G][e|E][r|R]\/[s|S]',
                r'[m|M][a|A][n|N][a|A][g|G][e|E][r|R]',
                r'[m|M][a|A][n|N][a|A][g|G][e|E][r|R][s|S]'
                ]

    new_is_value = re.sub('|'.join(new_is), 'immediate_supervisor_tag', text) # include _ in IS
    
    # Replace e.g.(manager[s], manager/[s], manager'[s])
    new_manager_value = re.sub('|'.join(new_manager), 'manager_tag', new_is_value)
    
    return new_manager_value

# CLEANUP PIPELINE

This cleanup pipelines is done using the functions above for cleanup. Returns separately list of docs subject for **topic modeling** and **sentiment analysis**.
* Using threshold = 99 for the `lang_detect`

For this we have to divide the pipeline since rows with empty string values will return an error if we would translate it. This is caused by feedbacks like `-`: only a special character 

In [ ]:
def cleanup_pipeline_1(texts):

###############SENTIMENT##################
  expressions_clean = rem_expressions(texts)

  paragraphs = get_paragraphs(expressions_clean)

  sentences = define_sentences(paragraphs)
  
  cl_structure = clean_structure(sentences)

  return cl_structure

After the texts has been translated, the match_nones functions is used since we would need to translate first tagalog to english since some feedbacks contains tagalog words like **na**.

In [ ]:
def cleanup_pipeline_2(texts):

##############TOPIC MODEL#################
  yn_vs = yn_variants(texts)

  null_vs = null_variants(texts)

  word_uniform = word_uniformity(texts)

  tl_spell = tl_spelling(word_uniform)

  translated = translate_lang(tl_spell, threshold = 99)

  detec_person = detect_person(translated, exclude_detect)

  return detec_person, yn_vs, null_vs

# Test on Q4

## Without id no. & Timestamp

In [ ]:
test_q4 = df['Q4'].to_list()

In [ ]:
q4 = pd.DataFrame({'q4_raw': test_q4})

In [ ]:
q4

,q4_raw
0,Leadership
1,Working / Supporting on weekends. The need to ...
2,The GCash p3 - the system
3,"Office Politics, corporate policy are not clea..."
4,I find the company's lack of structure and pro...
...,...
543,"No work life balance, people would still conta..."
544,System Owner responsibility and Company Cultur...
545,Internal Processes & Admin - One fo the core v...
546,"Workload, the fact that it was never a tech cu..."


In [ ]:
q4.dropna(inplace=True)

In [ ]:
q4

,q4_raw
0,Leadership
1,Working / Supporting on weekends. The need to ...
2,The GCash p3 - the system
3,"Office Politics, corporate policy are not clea..."
4,I find the company's lack of structure and pro...
...,...
543,"No work life balance, people would still conta..."
544,System Owner responsibility and Company Cultur...
545,Internal Processes & Admin - One fo the core v...
546,"Workload, the fact that it was never a tech cu..."


In [ ]:
q4['len'] = q4['q4_raw'].apply(lambda x: len(x.split()))

In [ ]:
q4

,q4_raw,len
0,Leadership,1
1,Working / Supporting on weekends. The need to ...,24
2,The GCash p3 - the system,6
3,"Office Politics, corporate policy are not clea...",11
4,I find the company's lack of structure and pro...,71
...,...,...
543,"No work life balance, people would still conta...",16
544,System Owner responsibility and Company Cultur...,10
545,Internal Processes & Admin - One fo the core v...,358
546,"Workload, the fact that it was never a tech cu...",30


In [ ]:
q4 = q4[q4.len != 1]
q4.drop(columns=['len'], inplace=True)
q4.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
q4

,q4_raw
0,Working / Supporting on weekends. The need to ...
1,The GCash p3 - the system
2,"Office Politics, corporate policy are not clea..."
3,I find the company's lack of structure and pro...
4,Everything is very manual
...,...
485,"No work life balance, people would still conta..."
486,System Owner responsibility and Company Cultur...
487,Internal Processes & Admin - One fo the core v...
488,"Workload, the fact that it was never a tech cu..."


In [ ]:
q4[20:25]

,q4_raw
20,The team has incompetent leaders and the team ...
21,there are employees who think they're entitled...
22,Insufficient tools for fraud investigation.
23,None so far.
24,The millennial mindset and view that micro man...


In [ ]:
q4[205:210]

,q4_raw
205,The Compliance team has taken a back seat in t...
206,Agile without backbone
207,"The lousy processes (or the lack thereof, or l..."
208,(pre-covid19) condition of the work environment.
209,Work-life Balance. Recognitions.


In [ ]:
type(q4[110:120]['q4_raw'].loc[117])

str

In [ ]:
q4['rem_exps'] = q4['q4_raw'].apply(lambda x: cleanup_pipeline_1(x))

<ipython-input-95-6566e6682486>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q4['rem_exps'] = q4['q4_raw'].apply(lambda x: cleanup_pipeline_1(x))


In [ ]:
q4[q4['rem_exps']=='']

,q4_raw,rem_exps
484,,


In [ ]:
q4.replace('', np.nan, inplace=True)
q4.dropna(subset=['rem_exps'], how='all', inplace=True) 
q4.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
q4

,q4_raw,rem_exps
0,Working / Supporting on weekends. The need to ...,Working or Supporting on weekends. The need to...
1,The GCash p3 - the system,The GCash p3 - the system.
2,"Office Politics, corporate policy are not clea...","Office Politics, corporate policy are not clea..."
3,I find the company's lack of structure and pro...,I find the company' lack of structure and proc...
4,Everything is very manual,Everything is very manual.
...,...,...
484,"No work life balance, people would still conta...","No work life balance, people would still conta..."
485,System Owner responsibility and Company Cultur...,System Owner responsibility and Company Cultur...
486,Internal Processes & Admin - One fo the core v...,Internal Processes & Admin - One fo the core v...
487,"Workload, the fact that it was never a tech cu...","Workload, the fact that it was never a tech cu..."


In [ ]:
q4['for_sentiment_analysis'], q4['match_nones'], q4['for_topic_model'] = zip(*q4['rem_exps'].map(lambda x: cleanup_pipeline_2(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
q4[q4['match_nones']=='has none']

,q4_raw,rem_exps,for_sentiment_analysis,match_nones,for_topic_model
23,None so far.,None so far.,None so far.,has none,None so far.
276,None so far,None so far.,None so far.,has none,None so far.
374,None that I can think of right now..,None that I can think of right now..,None that I can think of right now..,has none,None that I can think of right now..
458,None that I can recall,None that I can recall.,None that I can recall.,has none,None that I can recall.


In [ ]:
len(q4[q4['match_nones']=='has none'])

4

In [ ]:
q4.drop(columns=['rem_exps'], inplace=True)
q4.drop(q4[q4['match_nones'] == 'has none'].index, inplace = True)
q4.drop(columns=['match_nones'], inplace=True)
q4.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
q4

,q4_raw,for_sentiment_analysis,for_topic_model
0,Working / Supporting on weekends. The need to ...,Working or Supporting on weekends. The need to...,Working or Supporting on weekends. The need to...
1,The GCash p3 - the system,The GCash p3 - the system.,The GCash p3 - the system.
2,"Office Politics, corporate policy are not clea...","Office Politics, corporate policy are not clea...","Office Politics, corporate policy are not clea..."
3,I find the company's lack of structure and pro...,I find the company's lack of structure and pro...,I find the company's lack of structure and pro...
4,Everything is very manual,Everything is very manual.,Everything is very manual.
...,...,...,...
480,"No work life balance, people would still conta...","No work life balance, people would still conta...","No work life balance, people would still conta..."
481,System Owner responsibility and Company Cultur...,System Owner responsibility and Company Cultur...,System Owner responsibility and Company Cultur...
482,Internal Processes & Admin - One fo the core v...,Internal Processes & Admin - One of the core v...,Internal Processes & Admin - One of the core v...
483,"Workload, the fact that it was never a tech cu...","Workload, the fact that it was never a tech cu...","Workload, the fact that it was never a tech cu..."


In [ ]:
type(q4['q4_raw'].loc[24])

str

## With id no. & Timestamp

In [ ]:
q4_id = df[['DUMMY ID NO', 'Timestamp', 'Q4']]
q4_id

,DUMMY ID NO,Timestamp,Q4
0,D100088,2019-01-09 15:25:29,Leadership
1,D101431,2019-01-10 09:12:01,Working / Supporting on weekends. The need to ...
2,D100216,2019-01-11 14:50:57,The GCash p3 - the system
3,D100311,2019-01-11 16:29:58,"Office Politics, corporate policy are not clea..."
4,D100587,2019-01-17 13:07:40,I find the company's lack of structure and pro...
...,...,...,...
543,D101323,2022-08-12 12:59:27,"No work life balance, people would still conta..."
544,D100679,2022-08-16 14:13:33,System Owner responsibility and Company Cultur...
545,D101666,2022-08-17 21:11:26,Internal Processes & Admin - One fo the core v...
546,D100201,2022-08-19 10:49:24,"Workload, the fact that it was never a tech cu..."


In [ ]:
q4_id['Timestamp'] = q4_id['Timestamp'].astype('datetime64[ns]')

<ipython-input-105-4737760ccffb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q4_id['Timestamp'] = q4_id['Timestamp'].astype('datetime64[ns]')


In [ ]:
q4_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DUMMY ID NO  548 non-null    object        
 1   Timestamp    548 non-null    datetime64[ns]
 2   Q4           548 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 13.0+ KB


In [ ]:
q4_id['Timestamp'] = q4_id["Timestamp"].dt.to_period('M')

<ipython-input-107-1c4fd7db3d8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q4_id['Timestamp'] = q4_id["Timestamp"].dt.to_period('M')


In [ ]:
q4_id

,DUMMY ID NO,Timestamp,Q4
0,D100088,2019-01,Leadership
1,D101431,2019-01,Working / Supporting on weekends. The need to ...
2,D100216,2019-01,The GCash p3 - the system
3,D100311,2019-01,"Office Politics, corporate policy are not clea..."
4,D100587,2019-01,I find the company's lack of structure and pro...
...,...,...,...
543,D101323,2022-08,"No work life balance, people would still conta..."
544,D100679,2022-08,System Owner responsibility and Company Cultur...
545,D101666,2022-08,Internal Processes & Admin - One fo the core v...
546,D100201,2022-08,"Workload, the fact that it was never a tech cu..."


In [ ]:
q4_id.rename(columns={q4_id.columns[-1]: question_cols[int(q4_id.columns[-1][1:]) - 1]}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
q4_id.dropna(inplace=True)
q4_id.drop_duplicates(keep='first')
q4_id.rename(columns={'What did you like LEAST about your job and/or this Company?': 'q4_raw'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
q4_id

,DUMMY ID NO,Timestamp,q4_raw
0,D100088,2019-01,Leadership
1,D101431,2019-01,Working / Supporting on weekends. The need to ...
2,D100216,2019-01,The GCash p3 - the system
3,D100311,2019-01,"Office Politics, corporate policy are not clea..."
4,D100587,2019-01,I find the company's lack of structure and pro...
...,...,...,...
543,D101323,2022-08,"No work life balance, people would still conta..."
544,D100679,2022-08,System Owner responsibility and Company Cultur...
545,D101666,2022-08,Internal Processes & Admin - One fo the core v...
546,D100201,2022-08,"Workload, the fact that it was never a tech cu..."


In [ ]:
type(q4_id[110:120]['q4_raw'].loc[117])

str

In [ ]:
q4_id['rem_exps'] = q4_id['q4_raw'].apply(lambda x: cleanup_pipeline_1(x))

<ipython-input-113-76854f70fdd1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q4_id['rem_exps'] = q4_id['q4_raw'].apply(lambda x: cleanup_pipeline_1(x))


In [ ]:
q4_id[q4_id['rem_exps']=='']

,DUMMY ID NO,Timestamp,q4_raw,rem_exps
117,D101360,2019-11,-,
542,D100790,2022-08,,


In [ ]:
q4_id.replace('', np.nan, inplace=True)
q4_id.dropna(subset=['rem_exps'], how='all', inplace=True) 
q4_id.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
q4_id['for_topic_model'], q4_id['yn_variants'], q4_id['null_variants'] = zip(*q4_id['rem_exps'].map(lambda x: cleanup_pipeline_2(x)))

<ipython-input-116-4e915de73a6e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q4_id['for_topic_model'], q4_id['yn_variants'], q4_id['null_variants'] = zip(*q4_id['rem_exps'].map(lambda x: cleanup_pipeline_2(x)))


In [ ]:
q4_id[q4_id['null_variants']=='has null']

,DUMMY ID NO,Timestamp,q4_raw,rem_exps,for_topic_model,yn_variants,null_variants
24,D101582,2019-03,N/A,N/A.,n/a.,valid,has null
26,D100249,2019-03,None so far.,None so far.,none so far.,valid,has null
45,D100541,2019-06,None,None.,none.,valid,has null
61,D100474,2019-07,None,None.,none.,valid,has null
72,D101896,2019-08,None,None.,none.,valid,has null
101,D101268,2019-09,none,none.,none.,valid,has null
119,D100903,2019-11,nothing really,nothing really.,nothing really.,valid,has null
146,D101728,2020-01,none,none.,none.,valid,has null
157,D102065,2020-01,none,none.,none.,valid,has null
192,D102077,2020-04,Nothing,Nothing.,nothing.,valid,has null


In [ ]:
q4_id[q4_id['yn_variants']=='has yn']

,DUMMY ID NO,Timestamp,q4_raw,rem_exps,for_topic_model,yn_variants,null_variants


In [ ]:
q4_id

,DUMMY ID NO,Timestamp,q4_raw,rem_exps,for_topic_model,yn_variants,null_variants
0,D100088,2019-01,Leadership,Leadership.,leadership.,valid,valid
1,D101431,2019-01,Working / Supporting on weekends. The need to ...,Working / Supporting on weekends. The need to ...,working / supporting on weekends. the need to ...,valid,valid
2,D100216,2019-01,The GCash p3 - the system,The GCash p3 - the system.,the gcash p3 - the system.,valid,valid
3,D100311,2019-01,"Office Politics, corporate policy are not clea...","Office Politics, corporate policy are not clea...","office politics, corporate policy are not clea...",valid,valid
4,D100587,2019-01,I find the company's lack of structure and pro...,I find the company's lack of structure and pro...,i find the company's lack of structure and pro...,valid,valid
...,...,...,...,...,...,...,...
541,D101323,2022-08,"No work life balance, people would still conta...","No work life balance, people would still conta...","no work life balance, people would still conta...",valid,valid
542,D100679,2022-08,System Owner responsibility and Company Cultur...,System Owner responsibility and Company Cultur...,system owner responsibility and company cultur...,valid,valid
543,D101666,2022-08,Internal Processes & Admin - One fo the core v...,Internal Processes & Admin - One fo the core v...,internal processes & admin - one of the core v...,valid,valid
544,D100201,2022-08,"Workload, the fact that it was never a tech cu...","Workload, the fact that it was never a tech cu...","workload, the fact that it was never a tech cu...",valid,valid


In [ ]:
q4_id.drop(columns=['rem_exps'], inplace=True)
q4_id.drop(q4_id[q4_id['null_variants'] == 'has null'].index, inplace = True)
q4_id.drop(q4_id[q4_id['yn_variants'] == 'has yn'].index, inplace = True)
q4_id.drop(columns=['null_variants'], inplace=True)
q4_id.drop(columns=['yn_variants'], inplace=True)
q4_id.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
q4_id

,DUMMY ID NO,Timestamp,q4_raw,for_topic_model
0,D100088,2019-01,Leadership,leadership.
1,D101431,2019-01,Working / Supporting on weekends. The need to ...,working / supporting on weekends. the need to ...
2,D100216,2019-01,The GCash p3 - the system,the gcash p3 - the system.
3,D100311,2019-01,"Office Politics, corporate policy are not clea...","office politics, corporate policy are not clea..."
4,D100587,2019-01,I find the company's lack of structure and pro...,i find the company's lack of structure and pro...
...,...,...,...,...
499,D101323,2022-08,"No work life balance, people would still conta...","no work life balance, people would still conta..."
500,D100679,2022-08,System Owner responsibility and Company Cultur...,system owner responsibility and company cultur...
501,D101666,2022-08,Internal Processes & Admin - One fo the core v...,internal processes & admin - one of the core v...
502,D100201,2022-08,"Workload, the fact that it was never a tech cu...","workload, the fact that it was never a tech cu..."


In [ ]:
q4_id[(q4_id['for_topic_model'].str.contains('name_tag'))]

,DUMMY ID NO,Timestamp,q4_raw,for_sentiment_analysis,for_topic_model
101,D101961,2019-10,Dona Vasquez’ management style,name_tag' management style.,name_tag' management style.
133,D100170,2020-01,Team Silos; Team disconnected with the overal...,team name_tag; team disconnected with the over...,team name_tag; team disconnected with the over...
144,D100262,2020-01,I like the operation's Job's and the benefit'...,i like the operations name_tag and the benefit...,i like the operations name_tag and the benefit...
189,D101543,2020-05,Challenge/Delays in working with other departm...,challenge/delays in working with other departm...,challenge/delays in working with other departm...
221,D100421,2020-08,I dislike how the company has become very bure...,i dislike how the company has become very bure...,i dislike how the company has become very bure...
273,D100137,2021-04,Lack of employee empowerment / recognition/ ap...,lack of employee empowerment / recognition / a...,lack of employee empowerment / recognition / a...
337,D101749,2021-07,No JD for tribe leads yet,no name_tag for tribe leads yet.,no name_tag for tribe leads yet.
425,D100598,2022-03,micro manage in the form of Jira,micro manage in the form of name_tag.,micro manage in the form of name_tag.
432,D100699,2022-04,The hostile work environment brought about by ...,the hostile work environment brought about by ...,the hostile work environment brought about by ...
447,D100398,2022-05,everything is prio,everything is name_tag.,everything is name_tag.


In [ ]:
q4_id[(q4_id['for_topic_model'].str.contains('name_tag'))]

,DUMMY ID NO,Timestamp,q4_raw,for_topic_model
58,D101962,2019-07,"Unfair judgment. Crab mentality, people talkin...","unfair judgement. name_tag mentality, people t..."
101,D101961,2019-10,Dona Vasquez’ management style,name_tag' management style.
129,D101944,2019-12,1.discrimination: for about 4 months I was ask...,1.discrimination: for about 4 months i was ask...
189,D101543,2020-05,Challenge/Delays in working with other departm...,challenge/delays in working with other departm...
221,D100421,2020-08,I dislike how the company has become very bure...,i dislike how the company has become very bure...
273,D100137,2021-04,Lack of employee empowerment / recognition/ ap...,lack of employee empowerment / recognition / a...
432,D100699,2022-04,The hostile work environment brought about by ...,the hostile work environment brought about by ...
496,D100902,2022-08,"When I joined GCash, Karen's aware that my lon...","when i joined gcash, name_tag's aware that my ..."


In [ ]:
q4_id[(q4_id['for_topic_model'].str.contains('name_tag'))]

,DUMMY ID NO,Timestamp,q4_raw,for_topic_model
101,D101961,2019-10,Dona Vasquez’ management style,name_tag' management style.
144,D100262,2020-01,I like the operation's Job's and the benefit'...,i like the operations name_tag and the benefit...
189,D101543,2020-05,Challenge/Delays in working with other departm...,challenge/delays in working with other departm...
221,D100421,2020-08,I dislike how the company has become very bure...,i dislike how the company has become very bure...
273,D100137,2021-04,Lack of employee empowerment / recognition/ ap...,lack of employee empowerment / recognition / a...
432,D100699,2022-04,The hostile work environment brought about by ...,the hostile work environment brought about by ...
496,D100902,2022-08,"When I joined GCash, Karen's aware that my lon...","when i joined gcash, name_tag's aware that my ..."


In [ ]:
q4_id[(q4_id['for_topic_model'].str.contains('name_tag'))]

,DUMMY ID NO,Timestamp,q4_raw,for_topic_model
101,D101961,2019-10,Dona Vasquez’ management style,name_tag' management style.
189,D101543,2020-05,Challenge/Delays in working with other departm...,challenge/delays in working with other departm...
221,D100421,2020-08,I dislike how the company has become very bure...,i dislike how the company has become very bure...
273,D100137,2021-04,Lack of employee empowerment / recognition/ ap...,lack of employee empowerment / recognition / a...
432,D100699,2022-04,The hostile work environment brought about by ...,the hostile work environment brought about by ...
496,D100902,2022-08,"When I joined GCash, Karen's aware that my lon...","when i joined gcash, name_tag's aware that my ..."


# Export raw and processed feedback Q's

In [ ]:
new_df = df.copy()

In [ ]:
new_df.drop(columns=['year', 'month'], inplace=True)

In [ ]:
new_df.head(3)

,DUMMY ID NO,Timestamp,Name of your Group,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16
0,D101149,2022-08-23 12:01:42,PEOPLE & PURPOSE,"Family Circumstances, WORK SET UP & FLEXIBILITY",I had several conversations/discussions with G...,Very dynamic and fast paced - I was given the ...,Constant change in direction for certain proje...,TM should communicate the deadlines ahead of t...,Flexibility for remote/WFH work for working pa...,Option to add senior dependents through salary...,Gami is one of those leaders who has empathy &...,Nothing. I'm very happy with Gami as my IS :),In the beginning my role was confusing because...,Generally I feel like it lacks in transparency...,"Definitely. I was/am happy with my role, my re...",I really enjoy doing Talent Marketing for GCas...,I received regular feedback from Gami & Robert...,Regularly / frequently. Gami and I would sched...,None at the moment
1,D100418,2022-08-24 09:35:21,COMMERCIAL,Family Circumstances,- Tried to manage situation at home as so that...,Culture - my ideas are valued and heard even b...,Lack of control on items that would impact my ...,None,Need to fill headcount so that people are not ...,,Ferdie is a very good and smart leader. I thin...,I would check up more on the team and boost mo...,"Yes, had a ton of learning.",Average,Yes. Given that my issues at home are resolved...,,"Previously with my original IS, very frequentl...",Very few instances with new IS.,None
2,D101602,2022-08-26 21:20:32,FINANCE,"Compensation, Career Advancement Opportunity",Higher basic salary,"I am given the chance to give feedback, sugges...",Laptop. It takes a while before you get a lapt...,More company and team activities like outings ...,"I think, make sure that people have enough res...",retirement funds and carplan,My supervisor always checks up on us if we are...,None,Yes. I was given a permanent position and got ...,my supervisors are honest and treated me kindl...,Absolutely. I love working in Gcash.,It has been a blissful 5 years Gcash. I am and...,twice a year,Twice a year,None


In [ ]:
new_df.columns[3:]

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16'],
      dtype='object')

In [ ]:
def export_to_csv(df):
    for i, question in enumerate(df.columns[3:]):
        #Details
        print('Cleaning Question_' + str(i+1) + '...', question)

        #Extract feedbacks and clean it
        df_cleaned = df[['DUMMY ID NO', 'Timestamp', 'Name of your Group', question]]

        # Change the string to datetime and extract only the year & month
        df_cleaned['Timestamp'] = df_cleaned['Timestamp'].astype('datetime64[ns]')
        df_cleaned['Timestamp'] = df_cleaned["Timestamp"].dt.to_period('M')

        df_cleaned.rename(columns={df_cleaned.columns[-1]: 
                                  question_cols[int(df_cleaned.columns[-1][1:]) - 1]}, 
                          inplace=True) # Rename Columns

        n_question = df_cleaned.columns[-1] # Extract the renamed column

        df_cleaned.drop_duplicates(keep='first') # Drop duplicates
        df_cleaned.dropna(inplace=True) # Drop right away those null feedbacks

        # Apply the first cleanup pipeline
        df_cleaned['rem_exps'] = df_cleaned[n_question].apply(lambda x: cleanup_pipeline_1(x))

        df_cleaned.replace('', np.nan, inplace=True) #Replace empty string values with NaN
        df_cleaned.dropna(subset=['rem_exps'], how='all', inplace=True)  #Delete rows containing NaN or null values in `rem_exps` column
        df_cleaned.reset_index(drop=True, inplace=True) #Reset the index from lowest to highest

        # Apply the second cleanup pipeline
        df_cleaned['for_topic_model'], df_cleaned['yn_variants'], df_cleaned['null_variants'] = zip(*df_cleaned['rem_exps'].map(lambda x: cleanup_pipeline_2(x)))

        df_cleaned.drop(columns=['rem_exps'], inplace=True)
        df_cleaned.drop(df_cleaned[df_cleaned['null_variants'] == 'has null'].index, inplace = True)
        df_cleaned.drop(df_cleaned[df_cleaned['yn_variants'] == 'has yn'].index, inplace = True)
        df_cleaned.drop(columns=['null_variants'], inplace=True)
        df_cleaned.drop(columns=['yn_variants'], inplace=True)
        df_cleaned.reset_index(drop=True, inplace=True)

        #Make raw feedbacks and export it
        # df_raw = pd.DataFrame(values, columns=[question])
        df_raw = df[['DUMMY ID NO', question]]
        df_raw.to_csv(r'/content/drive/Shareddrives/R&D Innovations/HR Analytics/Topic Modeling/data/raw/Q'+str(i+1)+'_raw_b2.csv', index=False)

        print('*Raw Feedbacks Exported*')

        #Make processed feedbacks and export it
        df_cleaned.to_csv(r'/content/drive/Shareddrives/R&D Innovations/HR Analytics/Topic Modeling/data/cleaned/Q'+str(i+1)+'_cleaned_b2.csv', index=False)

        print('*Cleaned Feedbacks Exported*' + '\n')

In [ ]:
%%timeit -n1 -r1

export_to_csv(new_df)

Cleaning Question_1... Q1


<ipython-input-55-9c5c5622f969>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Timestamp'] = df_cleaned['Timestamp'].astype('datetime64[ns]')
<ipython-input-55-9c5c5622f969>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Timestamp'] = df_cleaned["Timestamp"].dt.to_period('M')
/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

*Raw Feedbacks Exported*
*Cleaned Feedbacks Exported*

Cleaning Question_2... Q2


<ipython-input-55-9c5c5622f969>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Timestamp'] = df_cleaned['Timestamp'].astype('datetime64[ns]')
<ipython-input-55-9c5c5622f969>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Timestamp'] = df_cleaned["Timestamp"].dt.to_period('M')
/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

*Raw Feedbacks Exported*
*Cleaned Feedbacks Exported*

Cleaning Question_3... Q3


...

<ipython-input-55-9c5c5622f969>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['rem_exps'] = df_cleaned[n_question].apply(lambda x: cleanup_pipeline_1(x))
/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

*Raw Feedbacks Exported*
*Cleaned Feedbacks Exported*

Cleaning Question_16... Q16


Token indices sequence length is longer than the specified maximum sequence length for this model (4182 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-55-9c5c5622f969>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['rem_exps'] = df_cleaned[n_question].apply(lambda x: cleanup_pipeline_1(x))
/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py:311: SettingWit

*Raw Feedbacks Exported*
*Cleaned Feedbacks Exported*

4min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<ipython-input-55-9c5c5622f969>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['for_topic_model'], df_cleaned['yn_variants'], df_cleaned['null_variants'] = zip(*df_cleaned['rem_exps'].map(lambda x: cleanup_pipeline_2(x)))
/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
for i, question in enumerate(new_df.columns[1:]):
  print(i, question)

0 What prompted you to leave the Company?
1 Before making a decision to leave, did you investigate other options that would enable you to stay?
2 What did you like MOST about your job and/or this Company?
3 What did you like LEAST about your job and/or this Company?
4 Is there anything you would like to suggest to make the Company's system effective?
5 What constructive comments would you give to Management to make this Company a better employer?
6 Are there any other benefits you feel should have been offered? Please provide details.
7 Do you think your Immediate Superior is a good leader? Why or Why not?
8 What would you have differently if you were the manager?
9 Were your objectives for joining this Company met? Please elaborate on how they were met or how they may have changed.
10 What were your feelings about the performance review process?
11 Would you work for this Company again if there is an opportunity? What circumstances/conditions (if there are any) will make you accept ag

In [ ]:
new_df.columns[1:]

Index(['What prompted you to leave the Company?',
       'Before making a decision to leave, did you investigate other options that would enable you to stay?',
       'What did you like MOST about your job and/or this Company?',
       'What did you like LEAST about your job and/or this Company?',
       'Is there anything you would like to suggest to make the Company's system effective?',
       'What constructive comments would you give to Management to make this Company a better employer?',
       'Are there any other benefits you feel should have been offered? Please provide details.',
       'Do you think your Immediate Superior is a good leader? Why or Why not?',
       'What would you have differently if you were the manager?',
       'Were your objectives for joining this Company met? Please elaborate on how they were met or how they may have changed.',
       'What were your feelings about the performance review process?',
       'Would you work for this Company again if there